使用許諾条件

①	設計一次エネルギー消費量算定法に基づく計算方法をPythonによりコーディングし、Jupyter notebook形式で保存したファイル（以下、本ファイルと呼ぶ）の著作権は、一般社団法人日本サステナブル建築協会に帰属します。
Copyright©2019. 一般社団法人日本サステナブル建築協会
All rights reserved. 

②	本ファイルは、住戸の設計一次エネルギー消費量算定方法に係る「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）2.1算定方法（第四章第四節 FF暖房機）」に示す計算方法に基づき、国土交通省国土技術政策総合研究所・国立研究開発法人建築研究所、株式会社ユーワークス、株式会社住環境計画研究所が作成したものです。

③	本ファイルは、計算方法に示す計算過程の確認及び調査研究等に活用することを目的として作成されたものであり、一般社団法人日本サステナブル建築協会は著作権について放棄はしませんが、使用する各人の自己責任において仕様を変更することを妨げるものではありません。ただし、本ファイルによる計算結果を住宅の省エネルギー基準で規定する設計一次エネルギー消費量の結果として公的な申請に用いることはできません。

④	本ファイルを変更するかしないかを問わず、以下の条件を満たす場合に限り、頒布および使用が許可されます。
1.	頒布物の付属ドキュメント等の資料に、上記の著作権表示、本条件一覧、および下記免責条項を含めること。
2.	書面による特別の許可なしに、本ファイルから派生した製品の宣伝または販売の促進に、一般社団法人日本サステナブル建築協会、国土技術政策総合研究所・国立研究開発法人建築研究所および株式会社ユーワークス、株式会社住環境計画研究所の名前またはコントリビューターの名前を使用してはならない。

⑤	本ファイルは、著作権者およびコントリビューターによって「現状のまま」提供されており、明示黙示を問わず、商業的な使用可能性、および特定の目的に対する適合性に関する暗黙の保証も含め、またそれに限定されない、いかなる保証もありません。著作権もコントリビューターも、事由のいかんを問わず、損害発生の原因いかんを問わず、かつ責任の根拠が契約であるか厳格責任であるか（過失その他の）不法行為であるかを問わず、仮にそのような損害が発生する可能性を知らされていたとしても、本ファイルの使用によって発生した（代用品または代用サービスの調達、使用の喪失、データの喪失、利益の喪失、業務の中断も含め、またそれに限定されない）直接損害、間接損害、偶発的な損害、特別損害、懲罰的損害、または結果損害について、一切の責任を負わないものとします。



In [ ]:
import nbimporter

import numpy as np
from s04_01_general import get_Q_T_H_d_t_i

## 第四節　FF暖房機 / Section 4  FF type heating

1．適用範囲～4．記号及び単位は省略

1. Scope of application ~ 4. Symbols and units are omitted

### 5．最大暖房出力 / Maximum heating output

$$\large
Q_{max,H,d,t} = q_{max,H} \times 3600 \times 10^{-6}
$$

$Q_{max,H,d,t}$：日付$d$の時刻$t$における1時間当たりの最大暖房出力(MJ/h) / Maximum heating output per hour at time $ t $ of date $ d $  
$q_{max,H}$：最大暖房能力(W) / Maximum heating capacity

In [ ]:
def get_Q_max_H_d_t(q_max_H):
    return np.ones(24 * 365) * q_max_H * 3600 * 10 ** (-6)  # (1)

### 6．暖房エネルギー消費量 / Heating energy consumption

#### 6.1　消費電力量 / Power consumption

$$
\large
E_{E,H,d,t} = \left\{ \begin{array}{ll}
    P_{rtd,H} \times \frac{Q_{T,H,d,t}}{Q_{max,H,d,t}} \times 10^{-3} & \small(Q_{T,H,d,t} \ge q_{min,H} \times 3600 \times 10^{-6}の場合)\\ 
    \left(P_{rtd,H} \times \frac{Q_{T,H,d,t}}{Q_{max,H,d,t}} + P_{itm,H} \right) \times 10^{-3} & \small(Q_{T,H,d,t} \lt q_{min,H} \times 3600 \times 10^{-6}の場合)\\ 
    \end{array}
\right.
$$

$E_{E,H,d,t}$：日付$d$の時刻$t$における1時間当たりの消費電力量(kWh/h) / Power consumption per hour at time $ t $ of date $ d $  
$q_{min,H}$：連続運転時最小能力(W) / Minimum capacity during continuous operation  
$P_{rtd,H}$：定格暖房消費電力(W) / Rated heating power consumption  
$P_{itm,H}$：断続時消費電力(W) / Intermittent power consumption  
$Q_{T,H,d,t}$：日付$d$の時刻$t$における1時間当たりの処理暖房負荷(MJ/h) / Processing heating load per hour at time $ t $ of date $ d $  
$Q_{max,H,d,t}$：日付$d$の時刻$t$における1時間当たりの最大暖房出力(MJ/h) / Maximum heating output per hour at time $ t $ of date $ d $  

日付$d$の時刻$t$における1時間当たりの処理暖房負荷$Q_{T,H,d,t}$が 0 の場合、$E_{E,H,d,t} = 0$とする。

If the processing heating load $ Q_{T, H, d, t} $ is 0 at time $ t $ of date $ d $, then $ E_{E, H, d, t} = 0 $ .

In [ ]:
def get_E_E_H_d_t(q_max_H, q_min_H, P_rtd_H, P_itm_H, L_H_d_t):
    # 最大暖房出力
    Q_max_H_d_t = get_Q_max_H_d_t(q_max_H)

    # 処理暖房負荷
    Q_T_H_d_t = get_Q_T_H_d_t_i(Q_max_H_d_t, L_H_d_t, None, '居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合')

    # 消費電力量
    tmp1 = P_rtd_H * Q_T_H_d_t / Q_max_H_d_t * 10 ** (-3)
    tmp1[Q_T_H_d_t < q_min_H * 3600 * 10 ** (-6)] = 0.0

    tmp2 = (P_rtd_H * Q_T_H_d_t / Q_max_H_d_t + P_itm_H) * 10 ** (-3)
    tmp2[Q_T_H_d_t >= q_min_H * 3600 * 10 ** (-6)] = 0.0

    E_E_H_d_t = tmp1 + tmp2

    # ただし、Q_T_H_d_tが0の場合は0
    E_E_H_d_t[Q_T_H_d_t == 0] = 0

    return E_E_H_d_t

#### 6.2　ガス消費量 / Gas consumption

日付$d$の時刻$t$における１時間当たりのガス消費量$E_{G,H,d,t}$は、FF暖房機の燃料がガスの場合日付$d$の時刻$t$における１時間当たりの燃料消費量$E_{F,H,d,t}$に等しいとし、FF暖房機の燃料が灯油の場合0とする。

Gas consumption $ E_{G, H, d, t} $ per hour at time $ t $ of date $ d $ is $ d $ at time $ t $ of date $ d $ when the fuel for the FF heater is gas It is assumed that the fuel consumption per hour is $ E_{F, H, d, t} $, and it is 0 when the fuel of the FF heater is kerosene.

In [ ]:
def get_E_G_H_d_t(fuel, q_max_H, e_rtd_H, L_H_d_t):
    if fuel == 'G':
        return get_E_F_H_d_t(q_max_H, e_rtd_H, L_H_d_t)
    elif fuel == 'K':
        return np.zeros(24 * 365)
    else:
        raise ValueError(fuel)

#### 6.3　灯油消費量 / Kerosene consumption

日付$d$の時刻$t$における１時間当たりの灯油消費量$E_{K,H,d,t}$は、FF暖房機の燃料が灯油の場合日付$d$の時刻$t$における１時間当たりの燃料消費量$E_{F,H,d,t}$に等しいとし、FF暖房機の燃料がガスの場合0とする。

The kerosene consumption $ E_{K, H, d, t} $ per hour at the time $ t $ of the date $ d $ is the time $ t $ of the date $ d $ when the fuel for the FF heater is kerosene It is assumed that the fuel consumption per hour is $ E_{F, H, d, t} $, and is 0 when the fuel of the FF heater is gas.

In [ ]:
def get_E_K_H_d_t(fuel, q_max_H, e_rtd_H, L_H_d_t):
    if fuel == 'K':
        return get_E_F_H_d_t(q_max_H, e_rtd_H, L_H_d_t)
    elif fuel == 'G':
        return np.zeros(24 * 365)
    else:
        raise ValueError(fuel)

#### 6.4　その他の燃料による一次エネルギー消費量 / Primary energy consumption by other fuels

日付$d$の時刻$t$における１時間当たりのその他の燃料による一次エネルギー消費量$E_{M,H,d,t}$は、0とする。

Primary energy consumption $ E_{M, H, d, t} $ by other fuel per hour at time $ t $ of date $ d $ is 0.

In [ ]:
def get_E_M_H_d_t():
    return np.zeros(24 * 365)

### 7．燃料消費量 / Fuel consumption

$$\large
E_{F,H,d,t} = \frac{Q_{T,H,d,t}}{e_{rtd,H}}
$$

$E_{F,H,d,t}$：日付$d$の時刻$t$における1時間当たりの燃料消費量(MJ/h) / Fuel consumption per hour at time $ t $ of date $ d $  
$Q_{T,H,d,t}$：日付$d$の時刻$t$における1時間当たりの処理暖房負荷(MJ/h) / Processing heating load per hour at time $ t $ of date $ d $  
$e_{rtd,H}$：定格燃焼効率 / Rated combustion efficiency  

In [ ]:
def get_E_F_H_d_t(q_max_H, e_rtd_H, L_H_d_t):
    # 最大暖房出力
    Q_max_H_d_t = get_Q_max_H_d_t(q_max_H)

    # 処理暖房負荷
    Q_T_H_d_t = get_Q_T_H_d_t_i(Q_max_H_d_t, L_H_d_t, None, '居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合')

    E_F_H_d_t = Q_T_H_d_t / e_rtd_H  # (3)

    return E_F_H_d_t


def get_Q_UT_H_d_t(q_max_H, L_H_d_t):
    # 最大暖房出力
    Q_max_H_d_t = get_Q_max_H_d_t(q_max_H)

    # 処理暖房負荷
    Q_T_H_d_t = get_Q_T_H_d_t_i(Q_max_H_d_t, L_H_d_t, None, '居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合')

    # 未処理暖房負荷
    Q_UT_H_d_t = L_H_d_t - Q_T_H_d_t

    return Q_UT_H_d_t

## 付録 A 機器の性能を表す仕様の決定方法 / Appendix A How to determine the specification that represents the performance of the device

### A.1　記号及び単位 / Symbol and unit

省略

ommision

### A.2　最大暖房能力 / Maximum heating capacity

$$\large
q_{max,H} = 240.1 \times A_{HCZ}
$$

$q_{max,H}$：最大暖房能力(W) / Maximum heating capacity  
$A_{HCZ}$：FF暖房機により暖房される暖冷房区画の床面積(m<sup>2</sup>) / Floor area of heating and cooling compartments heated by FF heaters  

In [ ]:
def get_q_max_H(A_HCZ):
    return 240.1 * A_HCZ  # (1)

### A.3　連続運転時最小能力 / Minimum capacity during continuous operation

連続運転時最小能力$q_{min,H}$が0を下回る場合は0とする。

When the continuous running minimum capacity $ q_{min, H} $ is less than 0, it is 0.

$$\large
q_{min,H} = 0.4334 \times q_{max,H} - 540.1
$$

$q_{min,H}$：連続運転時最小能力(W) / Minimum capacity during continuous operation

In [ ]:
def get_q_min_H(q_max_H):
    q_min_H = 0.4334 * q_max_H - 540.1  # (2)
    return max(0.0, q_min_H)

### A.4　定格燃焼効率 / Rated combustion efficiency

定格燃焼効率$e_{trd,H}$は0.860（86.0%）とするか、ガスを燃料にするものについては JIS S 2122 に定められた測定方法による熱効率又は灯油を燃料にするものについては JIS S 3031 に定められた測定方法による空気を暖める方式の機器の熱効率を確認し、1000 分の 1 未満の端数を四捨五入し小数第三位までの値とする。

The rated combustion efficiency $ e_{trd, H} $ is set to 0.860 (86.0%), or for those using gas as the fuel, thermal efficiency according to the measurement method defined in JIS S 2122, or those using kerosene as the fuel Check the thermal efficiency of the air heating system according to the measurement method specified in S3031 and round off fractions less than 1/1000 to obtain a value up to the third decimal place.

In [ ]:
def get_e_rtd_H_default():
    return 0.860


def get_e_rtd_H(e_rtd_H_raw):
    return round(e_rtd_H_raw * 1000) / 1000

### A.5　定格暖房消費電力 / Rated heating power consumption

$$\large
P_{rtd,H} = 3.13 \times 10^{-3} \times q_{max,H}
$$

$P_{rtd,H}$：定格暖房消費電力(W) / Rated heating power consumption

In [ ]:
def get_P_rtd_H(q_max_H):
    return 3.13 / 1000 * q_max_H  # (3)

### A.6　断続時消費電力 / Intermittent power consumption

断続時消費電力$P_{itm,H}$は、40.0Wとする。

The intermittent power consumption $ P_{itm, H} $ is 40.0 W.

In [ ]:
def get_P_itm_H():
    return 40.0

### A.7　複数のFF暖房機が設置される場合の仕様の決定方法 / How to determine the specifications when multiple FF heaters are installed

「主たる居室」に 2 台以上の暖房機が設置される場合、「その他の居室」に 2 台以上の暖房機が設置される場合又は複数の「その他の居室」においてそれぞれに暖房機が設置される場合は、「主たる居室」及び「その他の居室」それぞれにおいて、複数の機器の定格燃焼効率のうち最も小さい値を採用することとする。

When two or more heaters are installed in the "main room", two or more heaters are installed in the "other rooms", or a heater is installed in each of the "other rooms" In this case, in each of the “main room” and the “other room”, the smallest value among the rated combustion efficiencies of the plurality of devices is adopted.

In [ ]:
def get_e_rtd_from_multi_devices(e_rtd_list):
    return min(e_rtd_list)